# Compilación Just-in-time (JIT) con [Numba](http://numba.pydata.org/)

Podemos acelerar cálculos científicos de forma simple y semi-automática usando el compilador **Numba**. Este compilador no requiere cambiar el interprete de Python y tampoco es necesario aprender otro lenguaje

A través de decoradores podemos pedirle a Numba que compile una función "al vuelo" (just-in-time). Internamente Numba traduce las funciones de Python a lenguaje de máquina usando el compilador [LLVM](https://llvm.org/)

Para "ciertas funciones" el resultado será una versión compilada notoriamente más rápida en su ejecución que la original. Numba está diseñado para hacer más eficiente rutinas *compute-bound* que hagan **cálculos numéricos**. Tiene soporte para compilar funciones de Numpy y para paralelizar automaticamente ciclos `for`

Instalación utilizando conda:

    conda install numba

A continuación veremos los decoradores fundamentales de *Numba* y algunos ejemplos

Volvamos al código vectorizado para calcular la distancia euclidiana "todos con todos"

In [1]:
import numpy as np

data = np.random.randn(1000, 2)

def distancia_pares_numpy(data):
    return np.sqrt(np.sum((data.reshape(-1, 1, 2) - data.reshape(1, -1, 2))**2, axis=-1))

time_numpy = %timeit -r3 -n1 -o distancia_pares_numpy(data)

68.7 ms ± 5.64 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


Se utiliza numba para acelerar el cálculo de esta función:

- Importamos el decorador `jit` y lo aplicamos a la función anterior
- Usaremos el modo `nopython` o "modo rápido", esto indica al compilador que la función no usará el interprete de Python
- La primera llamada a la función es lenta, pues acciona el compilador
- La siguientes llamadas son más rápidas que la función de NumPy

In [2]:
from numba import jit

@jit(nopython=True)
def distancia_pares_numba(data):
    return np.sqrt(np.sum((data.reshape(-1, 1, 2) - data.reshape(1, -1, 2))**2, axis=-1))

distancia_pares_numba(data) # Aquí se ejecuta la compilación

time_numba = %timeit -r3 -n1 -o distancia_pares_numba(data)

20.1 ms ± 390 µs per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [3]:
time_numpy.average/time_numba.average

3.4113508284087355

In [4]:
np.allclose(distancia_pares_numpy(data), distancia_pares_numba(data))

True

- Existe un alias para `jit(nopython=True)` llamado `njit`
- Otros argumentos interesantes para decorar son: `parallel=True` y `fastmath=True` 